In [1]:
# https://github.com/python/cpython/blob/dc516ef8395d15da0ab225eb0dceb2e0581f51ca/Modules/_randommodule.c

N = 624
M = 397
MATRIX_A = 0x9908b0df
UPPER_MASK = 0x80000000
LOWER_MASK = 0x7fffffff

F = 1812433253

mag01 = [0x0, MATRIX_A]

MAX_BITS = 0b111111111111111111111111  # 32 bit


class Random:
    def __init__(self, seed):
        self.seed = seed
        self.state = [self.seed]
        
        for i in range(1, N):
            self.state.append(
                (F * (self.state[i - 1] ^ (self.state[i - 1] >> 30)) + i) & MAX_BITS
            )
            
        self.index = N
    
    def next(self):
        if self.index >= N:
            # Twist up state.
            
            for kk in range(N - M):  # 0 -> 227
                y = (self.state[kk] & UPPER_MASK) | (self.state[kk + 1] & LOWER_MASK)
                self.state[kk] = self.state[kk + M] ^ (y >> 1) ^ mag01[y & 0x1]


            for kk in range(N - M, N - 1):  # 227 -> 624
                y = (self.state[kk] & UPPER_MASK) | (self.state[kk + 1] & LOWER_MASK)
                self.state[kk] = self.state[kk + (M - N)] ^ (y >> 1) ^ mag01[y & 0x1]

            # 624
            y = (self.state[N - 1] & UPPER_MASK) | (self.state[0] & LOWER_MASK)
            self.state[N - 1] = self.state[M - 1] ^ (y >> 1) ^ mag01[y & 0x1]
            
            self.index = 0
        
        y = self.state[self.index]

        self.index += 1
        
        y ^= (y >> 11)
        y ^= (y << 7) & MAX_BITS & 0x9d2c5680 
        y ^= (y << 15) & MAX_BITS & 0xefc60000
        y ^= (y >> 18)

        return y

In [2]:
r = Random(160)
vals = [r.next() % 256 for _ in range(100_000)]

import statistics
print(statistics.mean(vals))
print(statistics.stdev(vals))

127.36043
73.86649932690955


In [3]:
import random

random.seed(5)
vals = [random.randint(0, 255) for _ in range(100_000)]
print(statistics.mean(vals))
print(statistics.stdev(vals))

127.74034
73.86581359647089


In [4]:
bin(2 ** 32 - 1)

'0b11111111111111111111111111111111'

In [5]:
len('11111111111111111111111111111111')

32